<a href="https://colab.research.google.com/github/apolitical/colab-repoclone/blob/master/colab_repoclone_testbed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

colab-repoclone-testbed
===================

A Google Colab notebook for verifying that the `colab-repoclone` module works as intended in the real world!

First you'll have to install the latest version of the library on [PyPI](https://pypi.org/project/colab-repoclone/) using `pip`.

In [1]:
!pip install colab-repoclone -qU

Importing the library should set everything up. If your Google Drive isn't already mounted, you'll have to complete an authentication flow. Otherwise it'll probably warn you that the drive is already mounted.

The `import` statement ought to have loaded everything in `colab_repoclone/__init__.py`. This includes a method, `colab_repoclone.local_repository()` for either cloning a specified GitHub repo into your Google Colab environment or initializing a new GitHub repo *from* your Google Colab environment.

If you don't have `vars.env` in your Google Drive, any empty one will be created at this point by the `colab_env` package. Otherwise, the variables will be loaded into the environment.

In [2]:
import colab_repoclone

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive




If all has gone to plan we ought to be able to access the module's `__version__`:

In [3]:
colab_repoclone.__version__

'0.2.3'

Now, we can try to clone or create a new repository using the `local_repository()` method. This method takes one input and three keywords.

The input is the link to the GitHub repository for cloning or the empty GitHub repo for initializing (you *must* have proper permissions if you wish to push to a repo).

The first keyword, `clone`, defaults to **True**, and the package will thus assume you are cloning an existing repository. If it is changed to **False** you must have a local directory that you wish to initialize and upload to GitHub as a new repository.

The second keyword, `branch`, allows the user to specify a branch to clone and defaults to "master" if none is indicated or if `clone=False`.

The third keyword, `auth_method`, specifies the method of authentication. This defaults to "env" which will look in the `vars.env` file loaded by `colab_env` for the following environment variables: `GITHUB_KEY`, `USER_NAME`, `USER_EMAIL` - i.e. your GitHub personal access token that gives you permission to edit the specified repo, your GitHub username and the email linked to your GitHub account. We can check the existence of these environment variables as follows:


In [4]:
import os
print(os.getenv("USER_NAME"),os.getenv("USER_EMAIL"))

# NB: I have not retrieved the GITHUB_KEY as Google Colab stores the output of
# cells and GitHub will not allow pushing of a file that contains a visible and
# valid GitHub access code in it.

jordanml7 lueck.jordan@gmail.com


If these environment variables do not exist it will throw an error indicating so. 

Alternatively, the `auth_method` keyword can be set to anything other than "env" and the user will instead be prompted to enter their valid GitHub username, email and authorization token.

## Cloning an existing repository

Let's try cloning an existing repository first. How about the `testbed` branch of this very repository?

When prompted to enter your various authentication strings you can just enter garbage - the repository will still clone but you will not have `push` permissions. Alternatively, replace the repo link with your own to confirm you can `push` to it!

In [5]:
my_clone = colab_repoclone.local_repository("https://github.com/apolitical/colab-repoclone",clone=True,branch="testbed",auth_method="input")

GitHub Username :: jordanml7
GitHub Email :: lueck.jordan@gmail.com
GitHub Authorization Token :: ··········


You can clone as many repositories as you want by calling the above command for each! The repo directories will be placed into the Google Colab `/content` directory and the methods for each can be accessed by referencing their respective object, such as `my_clone.push()` or `my_clone.pull()`.

Let's confirm that our repo name got set correctly:

In [6]:
my_clone.repo_dir

'colab-repoclone'

You can also confirm that the link to the repo on GitHub is correct by calling `my_clone.access_repo`, however this link contains our access token so I will not do so here.

Let's check that we properly pulled the `testbed` branch:

In [7]:
my_clone.branch

'testbed'

And while we're at it, let's confirm that our username and email got set correctly:

In [8]:
print(my_clone.github_user, my_clone.github_email)

jordanml7 lueck.jordan@gmail.com


Finally, although you've probably seen it in the `Files` sidebar by now (if not, click `REFRESH`), let's confirm that our repo is now in our Colab environment:

In [9]:
!ls /content/

colab-repoclone  gdrive  sample_data


There it is, `colab-repoclone` (or your own repo if you've changed it)! Wooo!

Now you can make any changes you want to the files in `colab-repoclone`, including adding and deleting files. Since this is an empty directory, let's create a dummy file that we'll throw in there:

In [0]:
dummy = open("dummy.txt","w+")

for i in range(10):
     dummy.write("This is test!\n")
    
dummy.close()

And now that we've made that *very* import file, let's move it into our repository:

In [0]:
!mv /content/dummy.txt /content/colab-repoclone/.

# Obviously you may have to change the path here if you're using your own repo

Let's confirm that it's in there:

In [12]:
!ls /content/colab-repoclone/

dummy.txt


There it is! We can now push our changes using `colab_repoclone`'s own `push` method!

If you're using the `colab-repoclone` repository, this will not work for you, as you do not have permission to push to it. Instead, you'll likely get a message saying:

`Command: < git push > failed. Check your permissions.`

But that's not a problem for me, because I *do* have permissions! The `push` method allows for two keywords: `commit_msg`, which is pretty self explanatory (if you do not supply one, you will be prompted for one), and `file_path`, which can be used to specify that you only want to push files in a certain directory *within* your repo. This defaults to all files, i.e. `"."`

Let's try it:

In [13]:
my_clone.push(commit_msg="Pushing to a repo from INSIDE the repo, meta...")


        *************************************************************
        * Are you sure you want to push your changes?               *
        *                                                           *
        * Press "q" to abort. Press any other key to continue...    *
        *************************************************************
        


Well, I didn't get any error messages, but you may have!

Now, because I don't want a stupid dummy file in my lovely empty repository (and because this tutorial relies on that file not existing initially), lets move `dummy.txt` out of this repo and push again!

In [0]:
!mv /content/colab-repoclone/dummy.txt /content/.

In [0]:
!ls /content/colab-repoclone/

And voila, it's gone.

In [16]:
my_clone.push(commit_msg="Annnddddd now its gone.")


        *************************************************************
        * Are you sure you want to push your changes?               *
        *                                                           *
        * Press "q" to abort. Press any other key to continue...    *
        *************************************************************
        


## Initializing a new repository

Now let's explore creating a new local directory, initializing it as a Git repo, and pushing it to GitHub.


First we have to create a local folder & put that dummy file from earlier into it.

In [0]:
!mkdir /content/my_new_repo
!mv /content/dummy.txt /content/my_new_repo/.

As before, we will use the `local_repository` method. The input is the link to the *existing* and yet empty repository on GitHub. I've made one on my personal GitHub account called `repo_init_testing`, that, again, you will not have push permissions for.

When you run the following command, you will be prompted for the name of the directory you wish to initialize. In this case, it's `my_new_repo`.

Since you do not have push permissions for my repo, unless you change it to one of your own, you will likely recieve the following warning upon executing this code:

`Command: < git push --set-upstream origin master > failed. Check your permissions.`

In [18]:
my_init = colab_repoclone.local_repository("https://github.com/jordanml7/repo_init_testing.git",clone=False)

# NB: This time I have not included the "branch" and "auth_method" keywords as 
# branch will always default to "master" for initializing a new repo and I
# already have my credentials set in my environment

Local Repository :: my_new_repo


Running the above will commit and push whatever files are in your new directory, `my_new_repo`, with the commit message "First Commit from Google Colab".

As above, you can now create new files, delete files, and modify existing files within this directory at will, then use the `push` method to upload any changes!

For now, let's remove the `dummy.txt` file we just added to our repository and push that changes.

In [0]:
!mv /content/my_new_repo/dummy.txt /content/.

In [20]:
my_init.push(commit_msg="Wooosshhh!")


        *************************************************************
        * Are you sure you want to push your changes?               *
        *                                                           *
        * Press "q" to abort. Press any other key to continue...    *
        *************************************************************
        


## Other features

The latest version off this package includes four methods not described above - let's explore those.

The first is `pull`, which allows you to pull changes to your repo if any were made elsewhere while you were running your notebook. Since all runtimes reset upon each load a notebook, you will need to "reclone" your repo each time you open and run your Colab notebook, so pulling will rarely be necessary.

We can try it on our clone `colab-repoclone` from earlier, but it won't do much since no new files were added to the `testbed` branch externally...

In [0]:
my_clone.pull()

We can also create a new branch after cloning or initializing a new repository, using the `new_branch` method.

This method takes a single keyword, the name of the branch you wish to create. If you do not enter one you will be prompted to do so. The specified branch is then created and checked-out. The correct upstream for future pushes from this branch is then set.

Let's try it:

In [22]:
my_clone.new_branch()

New Branch :: exciting_new_branch


Let's confirm that we're now in our new branch:

In [25]:
my_clone.branch

'exciting_new_branch'

*Very* exciting!

Now, because we love our `dummy.txt` file so much, let's add it to our new branch:

In [0]:
!mv /content/dummy.txt /content/colab-repoclone/.

In [26]:
my_clone.push(commit_msg="Branches 4 Dayz")


        *************************************************************
        * Are you sure you want to push your changes?               *
        *                                                           *
        * Press "q" to abort. Press any other key to continue...    *
        *************************************************************
        


That seemed to work for me! Again, the whole permissions mumbo-jumbo applies, so either accept your fate or replace all these repositories with your own!

Now let's get rid of it just to keep things clean...

In [0]:
!rm /content/colab-repoclone/dummy.txt

In [39]:
my_clone.push(commit_msg="Anybody feel like we have been here before?")


        *************************************************************
        * Are you sure you want to push your changes?               *
        *                                                           *
        * Press "q" to abort. Press any other key to continue...    *
        *************************************************************
        


This leads us nicely into the third cool new method, `checkout`. Pretty self explanatory. This method also takes a single keyword, the name of the *existing* branch you wish to checkout.

Let's checkout our `testbed` branch again:

In [40]:
my_clone.checkout()

Checkout Branch :: testbed


Let's confirm it worked...

In [41]:
my_clone.branch

'testbed'

Bazinga!

Last, but not least, we have `reset`. This method will perform a *hard* reset of your repository to a specific previous commit, passed as a keyword. If no commit ID is passed, it will default to rolling-back to the last commit.

Be careful with this one! It will wipe any changes you have made since the specified commit. And not just locally, but on GitHub as well :O

For demonstration purposes, I'm gonna roll our `testbed` repo waaayyyy back. A bunch of files will appear, old versions of what's currently in `master`.

In [54]:
my_clone.reset(commit="ca0ec747dbbed7da681b58d4c9fa6ca6cfd8c818")


        *****************************************************************
        *                         !! CAUTION !!                         *
        *                                                               *
        * Are you sure you want to rollback to a previous commit?       *
        *                                                               *
        * This is a hard reset, meaning all commits between the current *
        * and the one you are rolling back to will be lost.             *
        *                                                               *
        * Press "q" to abort. Press any other key to continue...        *
        *****************************************************************
        


In [61]:
!ls /content/colab-repoclone/

CHANGELOG.md	 dummy.txt  MANIFEST.in  README.md
colab_repoclone  LICENSE    Pipfile	 setup.py


Look at all those files! Where'd they come from? We're still in `testbed`...

In [62]:
my_clone.branch

'testbed'

... we've just rolled back to before all the bajillion pushes we made above *were* made!

Let's do some cleaning up of `testbed` (we'll use the `git rm` command as it won't mess with our `.git` file):

In [67]:
!git rm -r /content/colab-repoclone/*

rm 'CHANGELOG.md'
rm 'LICENSE'
rm 'MANIFEST.in'
rm 'Pipfile'
rm 'README.md'
rm 'colab_repoclone/__init__.py'
rm 'colab_repoclone/git_access.py'
rm 'setup.py'


And let's push our changes so `testbed` is all nice & empty:

In [68]:
my_clone.push(commit_msg="What a wild ride its been")


        *************************************************************
        * Are you sure you want to push your changes?               *
        *                                                           *
        * Press "q" to abort. Press any other key to continue...    *
        *************************************************************
        


And as some final cleaning up, let's just wipe all the new directories we've created:

In [0]:
!rm -rf colab-repoclone
!rm -rf my_new_repo

Well that was satisfying. Have fun and feel free to email me with questions or if you'd like to see any new functionality in a future release!